In [1]:
#To allocate memorey for gpu(here we are using 50%)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [4]:
# import the libraries as shown below

from keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [5]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Train/'
valid_path = 'Test/'

In [6]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

mobilnet = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 41s 1us/step


In [7]:
# don't train existing weights
for layer in mobilnet.layers:
    layer.trainable = False

In [9]:
# useful for getting number of output classes
folders = glob('Train/*')

In [10]:
folders

['Train\\Parasite', 'Train\\Uninfected']

In [11]:
# our layers - you can add more if you want
x = Flatten()(mobilnet.output)

In [12]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=mobilnet.input, outputs=prediction)

In [13]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [15]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [16]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 416 images belonging to 2 classes.


In [17]:
test_set = test_datagen.flow_from_directory('Test/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 134 images belonging to 2 classes.


In [18]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/50
13/13 [==============================] - 54s 4s/step - loss: 1.6010 - accuracy: 0.5288 - val_loss: 2.6777 - val_accuracy: 0.3284
Epoch 2/50
13/13 [==============================] - 6s 473ms/step - loss: 0.8898 - accuracy: 0.5962 - val_loss: 0.2553 - val_accuracy: 0.7687
Epoch 3/50
13/13 [==============================] - 6s 472ms/step - loss: 0.6160 - accuracy: 0.6995 - val_loss: 0.2637 - val_accuracy: 0.7985
Epoch 4/50
13/13 [==============================] - 6s 472ms/step - loss: 0.5163 - accuracy: 0.7404 - val_loss: 1.5323 - val_accuracy: 0.5597
Epoch 5/50
13/13 [==============================] - 10s 775ms/step - loss: 0.4532 - accuracy: 0.7885 - val_loss: 0.1572 - val_accuracy: 0.7239
Epoch 6/50
13/13 [==============================] - 5s 357ms/step - loss: 0.3614 - accuracy: 0.8389 - val_loss: 0.6831 - val_accuracy: 0.7687
Epoch 7/50
13/13 [==============================] - 5s 359ms/step - loss: 0.3015 - accuracy: 0.8846 - val_loss: 0.2926 - val_accuracy: 0.8284
Epoch 8

In [19]:
from keras.models import load_model
model.save('malaria_disease_vgg19.h5')